In [35]:
import rasterio
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
import hdbscan

In [36]:
with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/cropped.tif') as src:
    data = src.read()
    image = src.read(1)
    cols, rows = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    x = np.array(xs)
    x = (x-np.min(x))/(np.max(x)-np.min(x))
    y = np.array(ys)
    y = (y-np.min(y))/(np.max(y)-np.min(y))
    profile = src.profile

pixels = image.reshape(-1, 1)
pixels = np.clip(pixels, 0.5, pixels.max())
pixels = pixels/np.max(pixels)
print(pixels)

[[0.0001868 ]
 [0.00027317]
 [0.00043095]
 ...
 [0.0001868 ]
 [0.0001868 ]
 [0.0001868 ]]


In [37]:
data_points = []

for i in range(image.shape[0]):
    # Iterate through each column in the current row
    for j in range(image.shape[1]):
        data_points.append([x[i, j]*2, y[i, j]*2, image[i, j]])

data_points

[[0.0, 2.0, 0.1530791],
 [0.0026917900403543625, 2.0, 0.73117787],
 [0.005383580080754628, 2.0, 1.1535064],
 [0.008075370121108991, 2.0, 1.5225366],
 [0.010767160161509255, 2.0, 1.8091668],
 [0.013458950201863619, 2.0, 1.7170033],
 [0.016150740242263883, 2.0, 1.9818081],
 [0.018842530282618244, 2.0, 5.265824],
 [0.02153432032301851, 2.0, 6.0667105],
 [0.024226110363372876, 2.0, 1.779885],
 [0.026917900403773138, 2.0, 0.5725767],
 [0.029609690444127503, 2.0, 0.48162398],
 [0.032301480484527766, 2.0, 0.58805275],
 [0.03499327052488213, 2.0, 1.3008709],
 [0.0376850605652824, 2.0, 0.06651755],
 [0.040376850605636755, 2.0, 0.051792078],
 [0.04306864064603702, 2.0, 0.045932427],
 [0.045760430686391386, 2.0, 0.037258096],
 [0.04845222072679165, 2.0, 0.035282034],
 [0.05114401076714602, 2.0, 0.035599034],
 [0.053835800807546276, 2.0, 0.037491858],
 [0.05652759084790064, 2.0, 0.049648903],
 [0.05921938088830091, 2.0, 0.06912614],
 [0.06191117092865527, 2.0, 0.081985705],
 [0.06460296096905553, 

In [38]:
# with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/cropped.tif') as src:
#     image = src.read()

# pixels = image.reshape(image.shape[0], -1).T


In [39]:
print(image.shape)

(1200, 744)


In [40]:
# clstr = KMeans(50).fit(data_points)
# clstr = DBSCAN(eps=3,min_samples=500).fit(data_points)

# from multiprocessing import set_start_method, get_context
# set_start_method('spawn')

clusterer = hdbscan.HDBSCAN(min_cluster_size=150)
clstr= clusterer.fit_predict(data_points)

In [41]:
np.shape(clstr)
#removed.labels_ for hdbscan

(892800,)

In [42]:
unique_vals, counts = np.unique(clstr, return_counts=True)
#removed.labels_ for hdbscan
print(dict(zip(unique_vals, counts)))

{-1: 495293, 0: 1046, 1: 381, 2: 408, 3: 546, 4: 289, 5: 316, 6: 265, 7: 783, 8: 416, 9: 628, 10: 640, 11: 519, 12: 457, 13: 205, 14: 966, 15: 21315, 16: 200, 17: 2788, 18: 490, 19: 266, 20: 161459, 21: 465, 22: 385, 23: 383, 24: 239, 25: 357, 26: 749, 27: 575, 28: 380, 29: 447, 30: 428, 31: 299, 32: 311, 33: 388, 34: 931, 35: 369, 36: 595, 37: 311, 38: 392, 39: 187, 40: 549, 41: 190, 42: 427, 43: 431, 44: 872, 45: 1121, 46: 155, 47: 226, 48: 457, 49: 1827, 50: 345, 51: 157, 52: 178, 53: 273, 54: 378, 55: 158, 56: 633, 57: 168, 58: 214, 59: 360, 60: 312, 61: 536, 62: 166, 63: 184, 64: 210, 65: 189, 66: 251, 67: 407, 68: 332, 69: 5509, 70: 156, 71: 252, 72: 154, 73: 292, 74: 367, 75: 469, 76: 323, 77: 160, 78: 211, 79: 221, 80: 248, 81: 424, 82: 1228, 83: 277, 84: 165, 85: 229, 86: 166, 87: 1636, 88: 184, 89: 284, 90: 163, 91: 504, 92: 161, 93: 1087, 94: 164, 95: 492, 96: 1285, 97: 565, 98: 275, 99: 381, 100: 244, 101: 896, 102: 363, 103: 410, 104: 185, 105: 242, 106: 228, 107: 4790, 10

In [43]:
labels = clstr.reshape(image.shape[0], image.shape[1])
#removed.labels_ for hdbscan
np.shape(labels)

(1200, 744)

In [44]:
with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/clustered.tif', 'w+', driver='GTiff', width=image.shape[1], height=image.shape[0], count=2, dtype=labels.dtype, crs=src.crs, transform=src.transform) as dst:
    dst.write(image,1)
    # Write the cluster labels to a new band
    dst.write(labels,2)

# with rasterio.open('C:/Users/User/PycharmProjects/SafeRoute/clustered3.tif', 'w', driver='GTiff', width=image.shape[2], height=image.shape[1], count=1, dtype=np.uint8) as dst:
#     dst.write(labels_2d.astype(np.uint8), 1)
